TEAM 4 

In [1]:
import pandas as pd
import numpy as np
import regex as re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from wordcloud import WordCloud
import string
import pyLDAvis
from gensim import corpora
import warnings
import pyLDAvis.gensim_models  as gm
import gensim
import matplotlib.pyplot as plt
warnings.filterwarnings(action='once')

C:\Users\ktsmi\anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps
C:\Users\ktsmi\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
C:\Users\ktsmi\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.

In [3]:
tweet_data = pd.read_csv('JanDec2021.csv')

C:\Users\ktsmi\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
tweet_data.drop('Unnamed: 0',inplace = True,axis = 1)

In [5]:
tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250558 entries, 0 to 250557
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               250558 non-null  int64 
 1   conversation_id  250558 non-null  int64 
 2   created_at       250558 non-null  object
 3   date             250558 non-null  object
 4   time             250558 non-null  object
 5   user_id          250558 non-null  int64 
 6   username         250558 non-null  object
 7   name             250549 non-null  object
 8   place            251 non-null     object
 9   tweet            250558 non-null  object
 10  language         250558 non-null  object
 11  mentions         250558 non-null  object
 12  urls             250558 non-null  object
 13  photos           250558 non-null  object
 14  replies_count    250558 non-null  int64 
 15  retweets_count   250558 non-null  int64 
 16  likes_count      250558 non-null  int64 
 17  hashtags  

In [6]:
len(tweet_data)-sum(tweet_data.place.isnull())

251

In [7]:
#considering only english tweet
tweet_data = tweet_data[tweet_data.language == 'en']


In [8]:
tweet_data.reset_index(drop=True,inplace = True)

In [9]:
# clean the text
def cleantext(text):
    
    #remove any mentions
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text) 
    
    #remove hashtag
    text = re.sub(r'#', ' ', text)
    
    #removing html 
    text = re.sub(r'&\S+',' ',text)
    
    #remove hyperlink
    text = re.sub(r'https?://\S+',' ',text) 
    
    #keep only alpha  
    text = re.sub(r'[^a-zA-Z]',' ',text) 
    
    # convert to lower case
    words = [word.lower() for word in text.split()]   
    
    #filter out stop words 
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if not word in stop_words]
    
    #Lemmatisation
    lem = WordNetLemmatizer()
    words = [lem.lemmatize(word) for word in words]
    
    #consider words with length greatert than 1
    words = [word for word in words if len(word)>1]

    #remove custom stop words 
    custom_words = ['covid','india','case','coronavirus']
    words = [word for word in words if not word in custom_words]
    
    return words

In [10]:
#clean tweet
tweet_data['clean_tweet'] = tweet_data.tweet.apply(lambda x: cleantext(x))

In [11]:
#Extracrting corresponding states for India

state_list = ['Andhra Pradesh','Arunachal Pradesh','Assam','Bihar','Chhattisgarh','Goa','Gujarat','Haryana','Himachal Pradesh',
             'Jharkhand','Karnataka','Kerala','Madhya Pradesh','Maharashtra','Manipur','Meghalaya','Mizoram','Nagaland','Odisha',
             'Punjab','Rajasthan','Sikkim','Tamil Nadu','Telangana','Tripura','Uttar Pradesh','Uttarakhand','West Bengal']
state_list = [state.lower() for state in state_list]

#creating column for 28 states as dummy variable 
tweet_data[state_list] = 0

state_list_join = state_list
state_list_no_space = [''.join(state.split()) for state in state_list if len(state.split())>1 ]

state_list_join = state_list + state_list_no_space


#extracting place and adding a new column for the same.
tweet_data['state'] = tweet_data.clean_tweet.apply(lambda x: 
                                                   list(set(x).intersection(state_list_join)) if set(x).intersection(state_list_join) 
                                                  else '')

nospace_to_space_dict = {'andhrapradesh': 'andhra pradesh','arunachalpradesh': 'arunachal pradesh', 'himachalpradesh' : 'himachal pradesh','madhyapradesh':'madhya pradesh',
'tamilnadu':'tamil nadu', 'uttarpradesh':'uttar pradesh','westbengal':'west bengal'}

#adding 1 for corresponding state column
for ind, s_list in enumerate(tweet_data['state']):
    if(s_list):
        for s in s_list:
            if(s in nospace_to_space_dict.keys()):
                tweet_data.loc[ind,nospace_to_space_dict[s]]= 1
            else:
                tweet_data.loc[ind,s] = 1

In [12]:
#temp = tweet_data[state_list].sum().reset_index()
#temp.to_csv('state.csv',index = False)

In [13]:
len(tweet_data) - sum(tweet_data.state!='') , len(tweet_data) 

(209715, 221026)

In [14]:
#lets prepare custom stop words
#pd.Series(sum(tweet_data.clean_tweet[:3000].values,[])).value_counts()[:50]

In [15]:
#lets run word cloud to see if we want to remove unwanted commonly occuring word 
def show_wordcloud(data,title = None):
    wordcloud = WordCloud(
        background_color='white',
        max_words=200,
        max_font_size=40, 
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
    ).generate((" ").join(data))

    fig = plt.figure(1, figsize=(10, 10))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=1.35)

    plt.imshow(wordcloud)
    plt.show()

In [16]:
#show_wordcloud(tweet_data.clean_tweet.values.sum())

In [17]:
pd.Series(sum(tweet_data.clean_tweet[:5000].values,[])).value_counts()[:20]

vaccine        1554
new             818
news            632
time            600
dos             455
day             453
death           441
via             424
pm              402
people          399
positive        387
maharashtra     348
vaccination     348
modi            343
year            334
state           319
test            308
app             301
namo            288
record          285
dtype: int64

In [23]:
def lda_model(data,prefix,n_topics):
    corpus_df = data
    dictionary = corpora.Dictionary(corpus_df)
    corpus = [dictionary.doc2bow(text) for text in corpus_df]

    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = n_topics, id2word=dictionary, passes=15)
    ldamodel.save('model'+str(prefix)+'_'+str(n_topics)+'.gensim')
    topics = ldamodel.print_topics(num_words=4)

    for topic in topics:
        print(topic)
        
    lda_display = gm.prepare(ldamodel, corpus, dictionary, sort_topics=False)
    return(dictionary,corpus,pyLDAvis.display(lda_display))

In [24]:
lda_return = lda_model(tweet_data.clean_tweet,1,n_topics=3)
lda_return[2]

(0, '0.103*"vaccine" + 0.025*"vaccination" + 0.017*"approval" + 0.015*"pm"')
(1, '0.007*"time" + 0.006*"due" + 0.006*"year" + 0.005*"people"')
(2, '0.048*"new" + 0.038*"news" + 0.028*"death" + 0.022*"uk"')


In [26]:
#lda_model = gensim.models.ldamodel.LdaModel.load('model1_3.gensim')
#visualisation = gm.prepare(lda_model, lda_return[1], lda_return[0])
#pyLDAvis.save_html(visualisation, 'LDA_Visualization_1.html') 

In [29]:
#topic1 -> Covid cases and death 
#topic2 -> vaccine
#topic3 -> Income/money/finance  
#Extracted after analysis
topic_diction = {0:'covid cases and death',1:'vaccine',2:'income/money/finance'}


tweet_data['dominant_topic'] = ''
tweet_data['perc_contribution'] = 0
lda_decode = lda_model[lda_return[1]]

for ind, row in enumerate(lda_decode):
    top_val = sorted(row, key=lambda x: (x[1]), reverse=True)[0]
    
    tweet_data.loc[ind,'dominant_topic'] = topic_diction[top_val[0]]
    tweet_data.loc[ind,'perc_contribution'] = top_val[1]
    
#tweet_data.to_csv('tweet_topic_dataset.csv',index = False)